**Step 1: Data Loading and Preprocessing**

*1. We will first Load Data*

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load the dataset
file_path = 'IBM_2006-01-01_to_2018-01-01.csv'  # Adjust path if necessary
ibm_data = pd.read_csv(file_path)

# Select relevant columns
ibm_data = ibm_data[['Open', 'High', 'Low', 'Close']]


*2: Now we will Normalize Data*

In [3]:
# Initialize Min-Max Scaler
scaler = MinMaxScaler()

# Fit and transform the data
ibm_scaled = scaler.fit_transform(ibm_data)

# Convert to DataFrame for easier handling
ibm_scaled = pd.DataFrame(ibm_scaled, columns=ibm_data.columns)


**Step 2: Performing hyperparameter optimization for  LSTM**

In [ ]:
!pip install -q -U keras-tuner
import keras_tuner as kt
from tensorflow import keras

def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        return_sequences=True,
        input_shape=(X_train.shape[1], X_train.shape[2])
    ))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=50, step=32)))
    model.add(Dense(4))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='mean_squared_error'
    )
    return model

# Instantiate the tuner
tuner = kt.Hyperband(
    build_lstm_model,
    objective='val_loss',
    max_epochs=50,
    hyperband_iterations=2,
    directory='my_dir',
    project_name='keras_lstm'
)

# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=10, validation_split=0.1, verbose=1)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]



Reloading Tuner from my_dir/keras_lstm/tuner0.json

Search: Running Trial #87

Value             |Best Value So Far |Hyperparameter
512               |160               |units
0.0071098         |0.0064927         |learning_rate
50                |50                |tuner/epochs
0                 |17                |tuner/initial_epoch
0                 |3                 |tuner/bracket
0                 |3                 |tuner/round

Epoch 1/50
68/68 [==============================] - 33s 396ms/step - loss: 0.2310 - val_loss: 0.0357
Epoch 2/50
68/68 [==============================] - 25s 371ms/step - loss: 0.1033 - val_loss: 0.1176
Epoch 3/50
68/68 [==============================] - 37s 539ms/step - loss: 0.0953 - val_loss: 0.0061
Epoch 4/50
68/68 [==============================] - 25s 371ms/step - loss: 0.1217 - val_loss: 0.0945
Epoch 5/50
51/68 [=====================>........] - ETA: 6s - loss: 0.1107

**Step 2: Building LSTM and RNN Models**

*1. Preparing Time Series Data for LSTM/RNN Model*

In [13]:
import numpy as np

def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

# Define sequence length
seq_length = 10  # This is a hyperparameter

# Create sequences
X, y = create_sequences(ibm_scaled.values, seq_length)


2. Now we are building the LSTM Model

In [14]:
# Train the optimal LSTM model
history_optimal_lstm = optimal_lstm_model.fit(
    X_train, y_train,
    epochs=best_hps.get('epochs'),  # The number of epochs to train, as found by the tuner
    batch_size=32,  # Or you can use the batch size from the tuner
    validation_split=0.1,  # Using part of the training data as validation data
    verbose=1
)


NameError: ignored

*3. Now we are building the RNN Model*

In [6]:
from tensorflow.keras.layers import SimpleRNN

# Define RNN model
rnn_model = Sequential([
    SimpleRNN(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    SimpleRNN(50),
    Dense(4)  # Predicting 4 values (Open, High, Low, Close)
])

# Compile the model
rnn_model.compile(optimizer='adam', loss='mean_squared_error')


**Step 3: Training LSTM and RNN Models**

*1. Splitting Data into Training and Testing Sets*

As described in our model strategy

In [7]:
# Define a split for training and testing
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

# Split the data
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


*2. Now time for the Training LSTM Model*

In [8]:
# Train the LSTM model
history_lstm = lstm_model.fit(
    X_train, y_train,
    epochs=50,  # Number of epochs is a hyperparameter that can be tuned
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/50
68/68 [==============================] - 6s 27ms/step - loss: 0.0275 - val_loss: 9.6169e-04
Epoch 2/50
68/68 [==============================] - 1s 14ms/step - loss: 6.9720e-04 - val_loss: 0.0012
Epoch 3/50
68/68 [==============================] - 1s 13ms/step - loss: 6.8348e-04 - val_loss: 8.4196e-04
Epoch 4/50
68/68 [==============================] - 1s 14ms/step - loss: 7.2142e-04 - val_loss: 8.8964e-04
Epoch 5/50
68/68 [==============================] - 1s 20ms/step - loss: 6.6974e-04 - val_loss: 0.0011
Epoch 6/50
68/68 [==============================] - 1s 22ms/step - loss: 6.6538e-04 - val_loss: 0.0013
Epoch 7/50
68/68 [==============================] - 1s 21ms/step - loss: 6.8093e-04 - val_loss: 9.3255e-04
Epoch 8/50
68/68 [==============================] - 1s 18ms/step - loss: 6.8929e-04 - val_loss: 9.3832e-04
Epoch 9/50
68/68 [==============================] - 1s 15ms/step - loss: 6.3110e-04 - val_loss: 0.0013
Epoch 10/50
68/68 [==============================] - 1s 1

*3. Now we will start the Training RNN Model*

In [9]:
# Train the RNN model
history_rnn = rnn_model.fit(
    X_train, y_train,
    epochs=50,  # Number of epochs can be adjusted
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/50
68/68 [==============================] - 3s 13ms/step - loss: 0.0115 - val_loss: 0.0014
Epoch 2/50
68/68 [==============================] - 1s 8ms/step - loss: 8.0759e-04 - val_loss: 7.7581e-04
Epoch 3/50
68/68 [==============================] - 0s 7ms/step - loss: 6.1812e-04 - val_loss: 5.8218e-04
Epoch 4/50
68/68 [==============================] - 1s 9ms/step - loss: 5.5225e-04 - val_loss: 4.5217e-04
Epoch 5/50
68/68 [==============================] - 1s 11ms/step - loss: 5.1251e-04 - val_loss: 7.3816e-04
Epoch 6/50
68/68 [==============================] - 1s 12ms/step - loss: 4.7896e-04 - val_loss: 5.4089e-04
Epoch 7/50
68/68 [==============================] - 1s 12ms/step - loss: 3.9142e-04 - val_loss: 3.7427e-04
Epoch 8/50
68/68 [==============================] - 1s 13ms/step - loss: 3.7837e-04 - val_loss: 3.0806e-04
Epoch 9/50
68/68 [==============================] - 1s 13ms/step - loss: 4.4006e-04 - val_loss: 4.7957e-04
Epoch 10/50
68/68 [=============================

**Step 4: Evaluating and Plotting Model Performance**

*1. Plotting Training and Validation Loss*

After the training of the model, now we will try to plot the results

In [ ]:
import matplotlib.pyplot as plt

# Function to plot history
def plot_history(history, title):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Plot for LSTM
plot_history(history_lstm, 'LSTM Training and Validation Loss')

# Plot for RNN
plot_history(history_rnn, 'RNN Training and Validation Loss')
